In [26]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)
soundstream.load("content/soundstream/results3/soundstream.9000.pt")
soundstream.to("cuda")



SoundStream(
  (encoder): Sequential(
    (0): CausalConv1d(
      (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
    )
    (1): Sequential(
      (0): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (1): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (2): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(9,))
   

In [27]:
import sys
import torch
from importlib import reload
import network_models.soundstream_lstm.LSTM_dataset as lds
import torch.utils.data as data_utils
#from network_models.soundstream_lstm.LSTM_dataset  import AudioEmotionTessDataset
reload(lds)


tess_dataset = lds.AudioEmotionTessDataset("../tess")

encoded_dataset = lds.AudioEmotionTessSoundStreamEncodedDataset(dataSet= tess_dataset, soundStream=soundstream)

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/network_models/soundstream_lstm/LSTM_dataset.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensors.append(torch.nn.functional.pad(torch.tensor(data).to("cuda"), (0, 400 - data.shape[1], 0, 0)).detach())


In [28]:
import importlib
from pathlib import Path
import torch
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundstream_lstm.small_model as sm
importlib.reload(sm)


import network_models.soundstream_lstm.soundstream_datacollator as datacollator
importlib.reload(datacollator)
data_collator = datacollator.SoundStreamDataCollator(200)

# print(encoded_dataset.__getitem__(3).shape)
model = sm.EmotionClassifierSevenEmos().to("cuda")
tensor = torch.tensor(encoded_dataset.__getitem__(3)[0])
tensor = torch.nn.functional.pad(tensor, (0, 200 - tensor.shape[1], 0, 0))
tensor = tensor.flatten()
print(tensor[0].shape)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB (GPU 0; 9.77 GiB total capacity; 3.74 GiB already allocated; 1.17 GiB free; 5.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from torch.utils.data import Subset
import numpy as np
from network_models.soundstream_lstm.soundstream_datacollator import SoundstreamModelTrainer
from transformers import TrainingArguments, EvalPrediction

is_regression = False
from sklearn.model_selection import train_test_split

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    trainSet = Subset(dataset, train_idx)
    valSet = Subset(dataset, val_idx)
    return trainSet, valSet

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=0)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


train_dataset, eval_dataset = train_val_dataset(encoded_dataset, val_split=0.25)

training_args = TrainingArguments(
    output_dir="content/soundStreamClassModelsEmo",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=5.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)
trainer = SoundstreamModelTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#to override the collate_fn in dataloader because we have different sizes
def collate_fn(batch):
    return tuple(zip(*batch))

def pad_inputs(batch):
    input_batch = [torch.nn.functional.pad(torch.tensor(input_val).to("cuda"),
                        (0, 400 - input_val.shape[1], 0, 0))
                       for input_val in batch]
    return torch.stack((input_batch[0], input_batch[1]))

def pad_labels(batch):
    enc_list=[]
    for x in batch:
        a = np.zeros(7,)
        a[x] = 1
        enc_list.append(a)
    input_labels = [torch.tensor(label, dtype=torch.float32).to("cuda")
                    for label in enc_list]

    return torch.stack((input_labels[0], input_labels[1]))

In [ ]:
import torch.optim as optim

learning_rate = 1e-4
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
evalLoader = torch.utils.data.DataLoader(eval_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
train_dataset[1]

In [29]:
for epoch in range(300):
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        #get input values and labels from the batch of the dataloaded
        # print(data)
        inputs, labels = data[0].to("cuda"), data[1].to("cuda")

        # labels = data[1]
        #print(labels)
        #pad inputs
        # padded_inputs = pad_inputs(inputs)
        padded_labels = pad_labels(labels)
        #print(padded_inputs)
        #print(padded_labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        #print(outputs)
        loss = criterion(outputs.to(torch.float32), labels.to(torch.float32))
        #print(loss)
        loss.backward()
        optimizer.step()


        # print statistics
        running_loss += loss.item()

        # valid_loss = 0.0
        #
        # for i, data in enumerate(evalLoader, 0):
        #     inputs= data[0]
        #     labels = data[1]
        #     padded_inputs = pad_inputs(inputs)
        #     padded_labels = pad_labels(labels)
        #
        #     outputs = model(padded_inputs)
        #     loss = criterion(outputs, padded_labels)
        #     valid_loss = loss.item() * padded_inputs.size(0)
        #
        # print(valid_loss)

        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB (GPU 0; 9.77 GiB total capacity; 3.74 GiB already allocated; 1.17 GiB free; 5.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# trainer.train()
